In [1]:
import os, requests
import json, uuid
import arcpy

scratch = arcpy.env.scratchGDB
arcpy.env.overwriteOutput = True

In [8]:
def validation(pth):
    if arcpy.Exists(pth) == True:
        if int(arcpy.GetCount_management(pth).getOutput(0)) != 0:
            arcpy.Delete_management(pth)

def countOnly(url):
    response = requests.post(
        url,
        data={
            'where': '1=1',
            'outFields': "*",
            'returnCountOnly': 'true',
            'f': 'pjson'
        }
    )
    res = json.loads(response.text)
    cantidad = res["count"]
    return cantidad

def oidsOnly(url):
    lista_oids = []
    response = requests.post(
        url,
        data={
            'where': "1=1",
            'outFields': "*",
            'returnIdsOnly': 'true',
            'f': 'pjson'
        }
    )
    res = json.loads(response.text)
    lista_oids.extend(res["objectIds"])
    lista_oids = list(set(lista_oids))
    return(lista_oids)

def requestData(url, lista_oids):
    chunks = [lista_oids[x:x + 1000] for x in range(0, len(lista_oids), 1000)]
    for n in chunks:
        print(len(n))
        n = ','.join([str(x) for x in n])
        response = requests.post(
            url,
            data={
                'objectIds': n,
                'outFields': "*",
                'f': 'pjson'
            }
        )
        res = json.loads(response.text)
        feature = arcpy.AsShape(res, True)
        return feature

def uploadData(feature, pth):
    if arcpy.Exists(pth) == False:
        # arcpy.CopyFeatures_management(feature, pth)
        print("NO EXISTE O NO SE PUDO COPIAR")
    else:
        workspace = os.path.dirname(os.path.dirname(pth))
        edit = arcpy.da.Editor(workspace)
        edit.startEditing(False, True)
        arcpy.Append_management(feature, pth, 'NO_TEST')
        edit.stopEditing(True)

In [9]:
gdb = r'Database Connections\gdb.sde'
nameFd = r'gdb.sde.RiesgoDesastres'
nameFc = r'gdb.sde.CicatricesIncendios'
pth = os.path.join(gdb, nameFd, nameFc)
url = 'https://geo.serfor.gob.pe/geoservicios/rest/services/Visor/Incendios/MapServer/5/query'

In [10]:
print(pth)

Database Connections\gdb.sde\gdb.sde.RiesgoDesastres\gdb.sde.CicatricesIncendios


In [11]:
# Modelo de descargas
validation(pth)
cantidad = countOnly(url)
print("1\tExisten {} registros".format(cantidad))
lista_oids = oidsOnly(url)
print("2\tSe recuperaron los OIDs")
feature = requestData(url, lista_oids)
print("3\tSe descargaron los datos")
uploadData(feature, pth)
print("4\tSe subieron los datos")

1	Existen 38596 registros
2	Se recuperaron los OIDs
1000
3	Se descargaron los datos


RuntimeError: The operation was attempted on an empty geometry.

<RecordSet object at 0x1933cb90[0x19385800]>